# CS 198-120 HW 4

This homework is the jupyter notebook companion to HW4. For submission, please just export the notebook to a pdf and attach it to your written work for this homework.

This notebook will walk us through how to make your own quantum circuits in Qiskit using python. There are many ways to build circuits in qiskit sort of like how there are many notations for qubit states. I will be going over just one of these circuit notations which I think is the most clear and hides the least "under-the-hood". Here is a [reference for Qiskit's QuantumCircuit Module](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html) but the first part of this lab is how to read the circuit code! We'll be using the same skeletal structure for circuits throughout this notebook (and possibly future homeworks... if I can write them in time oops :>)

I'd like to note that most of Quantum Computing research (that I have been introduced to) doesn't deal with quantum computing in quite this way. Often there are layers of abstractions built upon the circuits to hide the messy circuit underneath. Like how in an electric circuit diagram you only see the Op-Amp, not all the components that build the Op-Amp. Of course, the reason for these exercises is to understand what is going on underneath those abstractions!

In [ ]:
"""
Author: Riley Peterlinz

Run this cell to import all the important goodies we'll be playing with.
"""

# always gotta have numpy
import numpy as np

# Various Qiskit Packages
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram
from qiskit.tools.visualization import circuit_drawer, array_to_latex
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector

"""
'sim' is the backend we'll be using to run the circuit. The aer_simulator is just doing standard matrix operations on your classical computer. You can also change this to one of the many quantum computers IBM has, but these have limited qubits, fidelity, and process time unfortunately. We're not quite at the point of making quantum computers a reality (yet!)
"""
sim = Aer.get_backend('aer_simulator')

def get_attributes(qc):
    """
    Helper Function!

    Given a quantum circuit, output the statevector and unitary of the circuit
    """
    qc_state = qc.remove_final_measurements(inplace=False).copy()
    qc_unitary = qc.remove_final_measurements(inplace=False).copy()

    qc_state.save_statevector()
    qvector = assemble(qc_state)
    state_vector = sim.run(qvector).result().get_statevector()

    qc_unitary.save_unitary()
    qunitary = assemble(qc_unitary)
    unitary = sim.run(qunitary).result().get_unitary()

    return state_vector, unitary

# How to make a Quantum Circuit in Qiskit

Here, we go through
1. How to initialize a quantum circuit in qiskit
2. How to add gates to a quantum circuit
3. How to do measurements on that circuit

## 1. Initializing a Quantum Circuit

Quantum Circuits can contain two types of registers: Quantum Registers and Classical Registers

The Quantum Registers are the registers (data lines) that hold qubits and we operate on with quantum gates

The Classical Registers are the standard binary bits we read the measurement of a quantum circuit to

Play around with the parameters of the QR and CR!

In [ ]:
qr = QuantumRegister(size=1, name='q') #QR takes two parameters (size, name) name is optional!
cr = ClassicalRegister(size=1 , name='cr') #CR takes two parameters (size, name) name is optional!
qc = QuantumCircuit(qr, cr) # Our quantum circuit takes the registers as inputs to create the circuit object

state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2) # the standard notation for drawing a circuit, scale gives higher resolutions

## 2. Adding Gates to the Circuit

Here, we will be showing how to make all the gates you need for this homework

The anatomy of adding a gate is that given your quantum circuit ```qc``` you can use dot notation to introduce a gate given some position in the circuit. For example, we can make the X gate (quantum version of the not gate) using
```python
qc.x(position)
```

There are two ways to specify the position:
1. You simply put in the number of the qubit lane you want the gate on. We index from 0 with the highest qubit lane being the 0th. i.e
```python
qc.x(0)
```
2. You can specify which quantum register you want to put it in and at what point in the register i.e.
```python
qc.x(qr[0])
```
places an x gate at the first position of the qr register. The code below shows how to implement the X, Y, Z, and CNOT gates as well as a variety of ways to specify position. The choice in position is up to you!

The basic dot notation for each gate is
- qc.x X-Gate
- qc.y Y-Gate
- qc.h Hadamard Gate
- qc.cnot CNOT Gate


In [ ]:
qx = QuantumRegister(size=1 , name='x')
qy = QuantumRegister(size=1 , name='y')
qz = QuantumRegister(size=1 , name='z')
qh = QuantumRegister(size=1 , name='h')
qcx = QuantumRegister(size=2 , name='cx')
qc = QuantumCircuit(qx, qy, qz, qh, qcx) # yes, you can have multiple registers in a single circuit!

"""
Gates
"""
qc.x(0) # X-Gate
qc.y(qy[0]) # Y-Gate
qc.z(qz[0]) # Z-Gate
qc.h(3) # Hadamard Gate
qc.cx(qcx[0], qcx[1]) # CNOT gate, the first parameter is the location of control, the second parameter is the location of the action

state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2) # the standard notation for drawing a circuit, scale gives higher resolutions

## 3. Taking Measurements

Measuring the quantum circuit means taking a certain amount of "shots" (aka measurements) from the circuit and observing their average. Let's walk through an example circuit to illustrate this.

The main difference is that we now apply qc.measure(qr[position], cr[position]) to the circuit

Running the code below should show these new meaurement blocks.

In [ ]:
qr = QuantumRegister(size=2, name='q')
cr = ClassicalRegister(size=2 , name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates
qc.x(qr[0])
qc.h(qr[1])
qc.x(qr[1])

"""
In order to measure a quantum register we must use the following method to apply the result of a quantum register to a classical register
"""
qc.measure(qr[0], cr[0]) # (QuantumRegister, ClassicalRegister)
qc.measure(qr[1], cr[1])

state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

For this state, the first qubit computes to
$$X\vert{0}\rangle = \vert{1}\rangle$$
and the second qubit computes to
$$HX\vert{0}\rangle = H\vert{1}\rangle = \vert{-}\rangle$$
So the combined state is
$$\vert{-}\rangle \otimes \vert{1}\rangle = \vert{-1}\rangle$$
Which, in terms of the $\vert{0}\rangle$, $\vert{1}\rangle$ basis
$$\vert{-}\rangle \otimes \vert{1}\rangle =
\frac{1}{\sqrt{2}}(\vert{0}\rangle - \vert{1}\rangle) \otimes \vert{1}\rangle =
\frac{1}{\sqrt{2}}(\vert{01}\rangle - \vert{11}\rangle)
$$
and vector representation is
$$\frac{1}{\sqrt{2}}(\vert{01}\rangle - \vert{11}\rangle) =
\frac{1}{\sqrt{2}}\begin{bmatrix} 0 \\ 1 \\ 0 \\ 1 \end{bmatrix}$$

We can verify this with the code below!


In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

Now, we can see the amplitudes for each state. Now remembering what we learned in problem 1, we can see that we measure the two states we consider in the analytical result and obtain their relative fractions, which is about equal to the complex square of the amplitude.

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

# Problem 2

There are 6 circuits to build and verify with your analytic calculations in this lab. I will provide the skeleton, but you must decide the number of registers needed, build the circuit, and run it. The answer for your statevector output should match the analytical output in your calculations.

Basically, edit the first cell of each problem and use the other two cells to evaluate your result!

## a) $\vert{01}\rangle$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

## b) $\vert{10}\rangle$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

## c) $\vert{+0}\rangle$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

## d) $\vert{-+}\rangle$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

## e) $\frac{1}{\sqrt{2}}(\vert{00}\rangle + \vert{01}\rangle)$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)

## f) $ \frac{1}{\sqrt{2}}(\vert{+10}\rangle + \vert{+11}\rangle)$

In [ ]:
qr = QuantumRegister(size=, name='q')
cr = ClassicalRegister(size=, name='cr')
qc = QuantumCircuit(qr, cr)

# defining gates

# define measurements
qc.measure

# helper methods, no need to edit!
state_vector, unitary = get_attributes(qc)
qc.draw(output = "mpl", scale = 2)

In [ ]:
display(array_to_latex(state_vector, prefix="\\text{Statevector} = ", max_size=30)) # display vector statevector
display(Statevector(state_vector).draw('latex')) # display 0,1 ket statevector
display(array_to_latex(unitary, prefix="\\text{Circuit = } ", max_size=30)) # display unitary

In [ ]:
# We run the simulator with sim.run(QUANTUM CIRCUIT) and we get the resulting values with .result()
result = sim.run(qc, shots=10000).result()

# We then collect the results using .get_counts()
counts = result.get_counts()

# Visualization
plot_histogram(counts)